<a href="https://colab.research.google.com/github/smw150430/NYPD-Frisk/blob/anesu/Visualizations_and_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler


In [2]:
#Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
path = "/content/drive/My Drive/cleaned_df.csv"
df = pd.read_csv(path, index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10474 entries, 0 to 11007
Data columns (total 63 columns):
MONTH2                                                          10474 non-null object
DAY2                                                            10474 non-null object
STOP_WAS_INITIATED                                              10474 non-null object
ISSUING_OFFICER_RANK                                            10474 non-null object
SUPERVISING_OFFICER_RANK                                        10474 non-null object
SUPERVISING_ACTION_CORRESPONDING_ACTIVITY_LOG_ENTRY_REVIEWED    10474 non-null object
JURISDICTION_DESCRIPTION                                        10474 non-null object
OBSERVED_DURATION_MINUTES                                       10474 non-null int64
SUSPECTED_CRIME_DESCRIPTION                                     10474 non-null object
STOP_DURATION_MINUTES                                           10474 non-null int64
OFFICER_EXPLAINED_STOP_FLAG      

In [0]:
df = df.drop(columns='DEMEANOR_CODE')


In [0]:
df = df.drop(['SEARCH_BASIS_INCIDENTAL_TO_ARREST_FLAG', 'SUSPECT_ARREST_OFFENSE'], axis=1)

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10474 entries, 0 to 11007
Data columns (total 60 columns):
MONTH2                                                          10474 non-null object
DAY2                                                            10474 non-null object
STOP_WAS_INITIATED                                              10474 non-null object
ISSUING_OFFICER_RANK                                            10474 non-null object
SUPERVISING_OFFICER_RANK                                        10474 non-null object
SUPERVISING_ACTION_CORRESPONDING_ACTIVITY_LOG_ENTRY_REVIEWED    10474 non-null object
JURISDICTION_DESCRIPTION                                        10474 non-null object
OBSERVED_DURATION_MINUTES                                       10474 non-null int64
SUSPECTED_CRIME_DESCRIPTION                                     10474 non-null object
STOP_DURATION_MINUTES                                           10474 non-null int64
OFFICER_EXPLAINED_STOP_FLAG      

In [0]:
X = df.drop('SUSPECT_ARRESTED_FLAG', axis = 1)
y = df.SUSPECT_ARRESTED_FLAG

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size= 0.2, stratify=y, random_state=345)

In [0]:
X_train_num = X_train[["OBSERVED_DURATION_MINUTES", "STOP_DURATION_MINUTES", "SUSPECT_HEIGHT", "SUSPECT_WEIGHT"]].columns
X_train_cat = X_train.drop(["OBSERVED_DURATION_MINUTES", "STOP_DURATION_MINUTES", "SUSPECT_HEIGHT", "SUSPECT_WEIGHT"], axis = 1).columns

In [61]:
X_train_cat.head()

,MONTH2,DAY2,STOP_WAS_INITIATED,ISSUING_OFFICER_RANK,SUPERVISING_OFFICER_RANK,SUPERVISING_ACTION_CORRESPONDING_ACTIVITY_LOG_ENTRY_REVIEWED,JURISDICTION_DESCRIPTION,SUSPECTED_CRIME_DESCRIPTION,OFFICER_EXPLAINED_STOP_FLAG,OTHER_PERSON_STOPPED_FLAG,SUMMONS_ISSUED_FLAG,OFFICER_IN_UNIFORM_FLAG,ID_CARD_IDENTIFIES_OFFICER_FLAG,SHIELD_IDENTIFIES_OFFICER_FLAG,VERBAL_IDENTIFIES_OFFICER_FLAG,FRISKED_FLAG,SEARCHED_FLAG,OTHER_CONTRABAND_FLAG,FIREARM_FLAG,KNIFE_CUTTER_FLAG,OTHER_WEAPON_FLAG,WEAPON_FOUND_FLAG,PHYSICAL_FORCE_CEW_FLAG,PHYSICAL_FORCE_DRAW_POINT_FIREARM_FLAG,PHYSICAL_FORCE_HANDCUFF_SUSPECT_FLAG,PHYSICAL_FORCE_OC_SPRAY_USED_FLAG,PHYSICAL_FORCE_OTHER_FLAG,PHYSICAL_FORCE_RESTRAINT_USED_FLAG,PHYSICAL_FORCE_VERBAL_INSTRUCTION_FLAG,PHYSICAL_FORCE_WEAPON_IMPACT_FLAG,BACKROUND_CIRCUMSTANCES_VIOLENT_CRIME_FLAG,BACKROUND_CIRCUMSTANCES_SUSPECT_KNOWN_TO_CARRY_WEAPON_FLAG,SUSPECTS_ACTIONS_CASING_FLAG,SUSPECTS_ACTIONS_CONCEALED_POSSESSION_WEAPON_FLAG,SUSPECTS_ACTIONS_DECRIPTION_FLAG,SUSPECTS_ACTIONS_DRUG_TRANSACTIONS_FLAG,SUSPECTS_ACTIONS_IDENTIFY_CRIME_PATTERN_FLAG,SUSPECTS_ACTIONS_LOOKOUT_FLAG,SUSPECTS_ACTIONS_OTHER_FLAG,SUSPECTS_ACTIONS_PROXIMITY_TO_SCENE_FLAG,SEARCH_BASIS_ADMISSION_FLAG,SEARCH_BASIS_CONSENT_FLAG,SEARCH_BASIS_HARD_OBJECT_FLAG,SEARCH_BASIS_OTHER_FLAG,SEARCH_BASIS_OUTLINE_FLAG,SUSPECT_SEX,SUSPECT_RACE_DESCRIPTION,SUSPECT_BODY_BUILD_TYPE,SUSPECT_EYE_COLOR,SUSPECT_HAIR_COLOR,STOP_LOCATION_PRECINCT,STOP_LOCATION_SECTOR_CODE,STOP_LOCATION_PATROL_BORO_NAME,STOP_LOCATION_BORO_NAME,age_bin
573,January,Monday,Based on Radio Run,POM,SGT,Y,PSB,CPW,Y,Y,N,Y,(null),(null),(null),Y,N,N,(null),(null),(null),N,(null),(null),(null),(null),(null),(null),Y,(null),Y,(null),(null),Y,Y,(null),(null),(null),(null),Y,(null),(null),(null),(null),(null),FEMALE,BLACK,MED,BRO,RED,103,C,PBQS,QUEENS,18_30
6832,August,Tuesday,Based on Self Initiated,POM,SGT,Y,PSB,CPW,Y,Y,N,N,(null),S,V,Y,N,N,(null),(null),(null),N,(null),Y,(null),(null),(null),(null),Y,(null),Y,(null),(null),(null),(null),(null),(null),(null),(null),Y,(null),(null),(null),(null),(null),MALE,BLACK HISPANIC,THN,BRO,BRO,44,E,PBBX,BRONX,18_30
5648,June,Friday,Based on Radio Run,POM,SGT,Y,PSB,MAKING GRAFFITI,Y,N,N,Y,(null),(null),(null),N,N,Y,(null),(null),(null),N,(null),(null),(null),(null),(null),(null),Y,(null),(null),(null),(null),(null),Y,(null),(null),(null),(null),Y,(null),(null),(null),(null),(null),MALE,BLACK,MED,BRO,BLD,49,C,PBBX,BRONX,over_30
2110,March,Tuesday,Based on Radio Run,POM,SGT,Y,Housing,OTHER,Y,N,N,N,(null),S,V,Y,N,N,(null),(null),(null),N,(null),Y,Y,(null),(null),(null),Y,(null),Y,Y,(null),(null),(null),(null),(null),(null),Y,Y,(null),(null),(null),(null),(null),MALE,BLACK,THN,BRO,BLK,43,D,PBBX,BRONX,18_30
4777,May,Thursday,Based on Radio Run,DTS,SGT,Y,PSB,ASSAULT,Y,N,N,Y,(null),(null),(null),N,N,N,(null),(null),(null),N,(null),(null),(null),(null),(null),(null),Y,(null),(null),(null),(null),(null),Y,(null),(null),(null),(null),(null),(null),(null),(null),(null),(null),FEMALE,BLACK,MED,ZZZ,BLK,73,D,PBBN,BROOKLYN,18_30


In [63]:


categorical_pipe = Pipeline([
  ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_pipe = Pipeline([
    ('scale', StandardScaler())
 ])

preprocessing = ColumnTransformer(
    [('cat', categorical_pipe, X_train_cat),
     ('num', numerical_pipe, X_train_num)])

rf = Pipeline([
    ('preprocess', preprocessing),
    ('classifier', RandomForestClassifier(random_state=42))
])
rf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocess',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('cat',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value='missing',
                                                                                 missing_values=nan,
                                                                                 strategy='constant',
                                                        

In [64]:
print("RF train accuracy: %0.3f" % rf.score(X_train, y_train))
print("RF test accuracy: %0.3f" % rf.score(X_test, y_test))

RF train accuracy: 1.000
RF test accuracy: 0.893


In [66]:
param_grid = { 
    'classifier__n_estimators': [200, 500],
    'classifier__max_features': ['auto', 'sqrt', 'log2'],
    'classifier__max_depth' : [4,5,6,7,8],
    'classifier__criterion' :['gini', 'entropy']}
from sklearn.model_selection import GridSearchCV
CV = GridSearchCV(rf, param_grid, n_jobs= 1)
                  
CV.fit(X_train, y_train)  
print(CV.best_params_)    
print(CV.best_score_)



{'classifier__criterion': 'entropy', 'classifier__max_depth': 8, 'classifier__max_features': 'auto', 'classifier__n_estimators': 500}
0.8777907598047946


In [68]:
categorical_pipe = Pipeline([
  ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_pipe = Pipeline([
    ('scale', StandardScaler())
 ])

preprocessing = ColumnTransformer(
    [('cat', categorical_pipe, X_train_cat),
     ('num', numerical_pipe, X_train_num)])

rf_new = Pipeline([
    ('preprocess', preprocessing),
    ('classifier', RandomForestClassifier(random_state=42, criterion='entropy', max_depth=8,max_features='auto', n_estimators=500))
])
rf_new.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocess',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('cat',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value='missing',
                                                                                 missing_values=nan,
                                                                                 strategy='constant',
                                                        

In [69]:
print("RF train accuracy: %0.3f" % rf_new.score(X_train, y_train))
print("RF test accuracy: %0.3f" % rf_new.score(X_test, y_test))

RF train accuracy: 0.893
RF test accuracy: 0.869


In [0]:
y_pred = rf_new.predict(X_test)

In [72]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[1435   55]
 [ 219  386]]
              precision    recall  f1-score   support

           N       0.87      0.96      0.91      1490
           Y       0.88      0.64      0.74       605

    accuracy                           0.87      2095
   macro avg       0.87      0.80      0.83      2095
weighted avg       0.87      0.87      0.86      2095

0.8692124105011934


In [73]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    NuSVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
    ]
for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
    pipe.fit(X_train, y_train)   
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))

NameError: ignored